In [ ]:
import sys
import os
import numpy as np
from source.model.incremental import RecursiveLeastSquares
from source.detector.cusum import CUSUM_Detector
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
n = 600

# Create time series components
t = np.arange(n)

# Trend component (slowly increasing)
trend = 0.05 * t

# Seasonal component (yearly pattern)
seasonal = 10 * np.sin(5 * np.pi * t / 365)

# Random noise
noise = np.random.normal(0, 5, n)

# Combine components
values = 50 + trend + seasonal + noise

# Abrupt shift on data

In [ ]:
values[300:500] += 50  # Introduce a change point

# plot the time series
plt.figure(figsize=(15, 4))
plt.plot(t, values, label='Time Series Data')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Synthetic Time Series with Change Point')
plt.legend()
plt.show()

# Monitoring ml-model performance on data shift

In [ ]:
# Set number of lags
num_lags = 3

# Initialize model
model = RecursiveLeastSquares(num_variables=num_lags, forgetting_factor=0.99, initial_delta=50)

warmup_period = 100

# Detect change points using CUSUM Detector
cusum_detector = CUSUM_Detector(warmup_period=warmup_period, delta=3, threshold=5)

# Store predictions and observed values
list_predictions = []
list_observed = []
list_residuals = []

list_pos_changes = []
list_change_points = []
list_index_changes = []
list_output_detected = []

# Create a figure for the animation with grid layout
fig, axs = plt.subplots(3, 1, figsize=(20, 15))
prediction_ax, residual_ax, change_ax = axs
fig.tight_layout(pad=5.0)

# Animation function
def animate(i):

    global output_detected, list_change_points, list_output_detected, list_residuals
    
    # Ensure we don't go out of bounds
    if i >= len(values) - num_lags - 1:
        return
    
    # Get the input
    X = values[i:i + num_lags].reshape(-1, 1)

    # Make a prediction
    y_pred = model.predict(X)

    # Get the true value
    y = values[i + num_lags + 1].reshape(-1, 1)

    if i > 20:  # wait for residual to stabilize

        # Store prediction and observed values
        list_predictions.append(y_pred)
        list_observed.append(y[0])

        # model params update
        model.update(X, y)

        # Compute residuals
        residuals = np.abs(y - y_pred)
        list_residuals.append(residuals[0])

        # Detect change points
        output_detected = cusum_detector.detection(residuals[0])
        list_pos_changes.append(output_detected[0][0])
        list_change_points.append(output_detected[-1])
        list_output_detected.append(output_detected)

        # Clear the axes for new plots
        prediction_ax.clear()
        residual_ax.clear()
        change_ax.clear()

        # Plot predictions vs observed values
        prediction_ax.plot(list_observed, label='Obs', color='blue')
        prediction_ax.plot(list_predictions, label='Pred', color='orange')
        for i, is_cp in enumerate(list_change_points):
            if is_cp:
                prediction_ax.axvline(x=i, color='red', linestyle='--', alpha=0.5)
        if i >= warmup_period:
            prediction_ax.axvline(x=warmup_period, color='gray', label='End WarmUp', linestyle='--')
        prediction_ax.plot()
        prediction_ax.legend()
        prediction_ax.set_title(f'Change Points at idx {100} and {500}\n RLS Predictions vs Observed Values')
        
        # Plot residuals
        residual_ax.plot(list_residuals, label='Residuals', color='red')
        residual_ax.legend()
        residual_ax.set_title('Residuals')

        # Plot detected change points
        change_ax.plot(list_pos_changes, label='Performance Cumulative Changes', color='green')
        change_ax.axhline(y=cusum_detector.threshold, color='r', linestyle='--')
        change_ax.legend()
        change_ax.set_title('Detected Change Points')

# Create the animation
ani = FuncAnimation(fig, animate, frames=len(values) - num_lags - 1, repeat=False)  #

# Save the animation
ani.save('monitoring.gif', writer='pillow', fps=100)
plt.close(fig)  # Prevent the figure from displaying